<a href="https://colab.research.google.com/github/kayaatub/api-youtube/blob/main/YouTube%20API%20Practice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests 
import pandas as pd
import time
import plotly

In [21]:
api_key1 = "in the email"
channel_id1 ="UC46vj6mN-6kZm5RYWWqebsg"
channel_id2 = "UCwZUYWxwvU8RxcDPAggbhdw"

In [5]:
#import libraries
import requests
import pandas as pd
import time

In [13]:
#Keys
API_KEY = api_key1
CHANNEL_ID = channel_id1

In [7]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, dislike_count, comment_count

In [8]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"dislike_count":dislike_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [16]:
#main

#build our dataframe
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"]) 

df2 = get_videos(df2)

In [17]:
df2

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,OdEZbFxsnks,Tips For How To Get Freelance Projects in Soft...,2021-08-11,30488,845,8,76
1,rpZcTKJTOi4,Demo Websites for practicing Rest API Testing,2021-08-09,14801,401,2,21
2,mcQI5x2T1RA,Demo Websites Projects for practicing Automat...,2021-08-07,21070,732,4,79
3,KFK9-pK7N2I,How To Improve Coding Skills | Tips And Tricks,2021-08-05,16322,498,5,25
4,pqq-oUc0Wlw,Tips for getting your CV shortlisted and get m...,2021-08-03,13095,444,2,54
...,...,...,...,...,...,...,...
492,04ulACGXGas,Python Tutorial 16 - List Vs Dictionary Vs Tup...,2018-09-16,5021,66,2,5
493,-WSkxYC3xiA,Python Tutorial 15 - Tuples in Python,2018-09-16,2302,23,0,0
494,MLt-1KonDR4,Python Tutorial 11 - Numbers in Python,2018-09-12,2928,27,2,0
495,Kt4DVlR4NKo,Software Testing - Real Time Interview Questio...,2018-09-06,267163,4334,138,153


In [19]:
df2.to_csv('youtube_vids_2nd_pull.csv')